In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Reshape

import numpy as np
import pandas as pd

np.random.seed(10)

print (tf.__version__)

In [2]:
data = pd.read_csv('colors.csv')
print(data)

                  name  red  green  blue
0             parakeet  174    182    87
1         saddle brown   88     52     1
2       cucumber crush  222    237   215
3            pool blue  134    194   201
4             distance   98    110   130
...                ...  ...    ...   ...
14152        fire mist  251    216   197
14153  spanish leather  143    102    54
14154         aquatone  165    180   170
14155     bottle green    9     54    36
14156   tandoori spice  172    112   107

[14157 rows x 4 columns]


In [3]:
print(data.head(5))

             name  red  green  blue
0        parakeet  174    182    87
1    saddle brown   88     52     1
2  cucumber crush  222    237   215
3       pool blue  134    194   201
4        distance   98    110   130


In [4]:
len(data)

14157

In [5]:
names = data["name"]
print(names)

0               parakeet
1           saddle brown
2         cucumber crush
3              pool blue
4               distance
              ...       
14152          fire mist
14153    spanish leather
14154           aquatone
14155       bottle green
14156     tandoori spice
Name: name, Length: 14157, dtype: object


In [6]:
h = sorted(names.str.len().values)

import matplotlib.pyplot as plt
import scipy.stats as stats

fit = stats.norm.pdf(h, np.mean(h), np.std(h))
plt.plot(h,fit,'-o')
plt.hist(h, density=True)
plt.xlabel('Chars')
plt.ylabel('Probability density')
plt.show()

KeyboardInterrupt: 

In [ ]:
np.array(h).max()

In [ ]:
maxlen = 25
t = Tokenizer(char_level = True)
t.fit_on_texts(names)
tokenized = t.texts_to_sequences(names)
padded_names = preprocessing.sequence.pad_sequences(tokenized,maxlen = maxlen)

In [ ]:
from keras.utils import to_categorical
one_hot_names = to_categorical(padded_names)
num_classes = one_hot_names.shape[-1]
num_classes

In [ ]:
print(one_hot_names)
print(one_hot_names.shape)
t.word_index

In [ ]:
padded_names

In [ ]:
# converting rgb values from 0-255 to 0-1
def norm(value):
    return value/255
normalized_values = np.column_stack([norm(data["red"]),norm(data["green"]),norm(data["blue"])])
print(normalized_values)

In [ ]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(maxlen, num_classes)))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(optimizer='adam', loss='mse', metrics=['acc'])
model.summary()

In [ ]:
normalized_values.shape

In [ ]:
one_hot_names.shape

In [ ]:
history = model.fit(one_hot_names, normalized_values,
                    epochs=40, 
                    batch_size=32,
                    validation_split=0.1)

In [ ]:
# train with all data including the 10% that was not included before
model.fit(one_hot_names, normalized_values,
          epochs=10,
          batch_size=32)

In [ ]:
model.save_weights('model_1.weights.h5')

In [10]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation acc')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'history' is not defined

In [8]:
# fn. for plot color image
def plot_rgb(rgb):
    data =[[rgb]]
    plt.figure(figsize=(2,2))
    plt.imshow(data, interpolation='nearest')
    plt.show()

# fn. for rescaling again to 0-255
def scale(n):
    return int(n*255)

# final fn. to display the color
def predict(name):
    name = name.lower()
    tokenized = t.texts_to_sequences([name])
    padded = preprocessing.sequence.pad_sequences(tokenized, maxlen=maxlen)
    from keras.utils import to_categorical
    one_hot = to_categorical(padded, num_classes=num_classes)
    pred = model.predict(np.array(one_hot))[0]
    r, g, b = scale(pred[0]), scale(pred[1]), scale(pred[2])
    print(name + ',', 'R,G,B:', r,g,b)
    plot_rgb(pred)

In [9]:
predict("lipstick")
predict("mauve")
predict("keras red")
predict("bull cream")

NameError: name 't' is not defined